In [1]:
import findspark

In [5]:
findspark.init('/home/admin123/spark-2.4.0-bin-hadoop2.7')

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName('LgReg').getOrCreate()

In [8]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [9]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [10]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [41]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [42]:
output = assembler.transform(data)

In [43]:
final_data = output.select('features', 'churn')

In [44]:
train_churn,test_churn = final_data.randomSplit([0.7, 0.3])

In [45]:
from pyspark.ml.classification import LogisticRegression

In [46]:
lr_churn = LogisticRegression(labelCol='churn')

In [47]:
fit_churn_model = lr_churn.fit(train_churn)

In [49]:
train_summary = fit_churn_model.summary

In [50]:
train_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                647|                647|
|   mean|  0.160741885625966|0.12364760432766615|
| stddev|0.36757687828155483| 0.3294337755409456|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [52]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [59]:
pred_and_labels = fit_churn_model.evaluate(test_churn)

In [60]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,11128.95,1....|    0|[4.35031169161134...|[0.98726157101529...|       0.0|
|[28.0,11204.23,0....|    0|[1.82380042674718...|[0.86102152352288...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.94402957357500...|[0.94998054965129...|       0.0|
|[29.0,11274.46,1....|    0|[4.64835271846173...|[0.99051349021125...|       0.0|
|[29.0,13240.01,1....|    0|[6.82297959865026...|[0.99891271061156...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.35631612939531...|[0.96631105779661...|       0.0|
|[30.0,10744.14,1....|    1|[1.92799095291228...|[0.87302688196008...|       0.0|
|[30.0,12788.37,0....|    0|[2.42230377538396...|[0.91851234251568...|       0.0|
|[31.0,7073.61,0.0...|    0|[3.19773631266486...|[0.96074900172279...|       0.0|
|[32.0,8011.38,0

In [62]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [64]:
#caluculation accuracy using BinaryClassificationEvaluator
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [65]:
auc

0.7282608695652175

In [69]:
#Prediction on New Data
final_lr_model = lr_churn.fit(final_data)

In [70]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [71]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [79]:
test_new_customer = assembler.transform(new_customers)

In [80]:
test_new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [82]:
final_result = final_lr_model.transform(test_new_customer)

In [85]:
#predicting with company labels
final_result.select('company', 'prediction').show()

+----------------+----------+
|         company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [86]:
test_new_customer.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          